<a href="https://colab.research.google.com/github/ErikFantomex/MPI_Curso/blob/master/Intro_laboratorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tarea 3 : Primos gemelos

Dos números primos, p1 y p2, son gemelos si la diferencia entre ellos es igual a 2. 

Suponer que se tienen p procesos y un número n=2,000,000 (o mayor)

Determinar el número de parejas de primos gemelos menores o iguales a n, dividiendo el trabajo entre los p procesos. Hacer que uno de los procesos envíe a pantalla el valor resultante. 


VERSION 1 
1. Utilizar MPI_Send() y MPI_Recv() para las comunicaciones. Que cada proceso le envía el numero de pares de primos gemelos que encontró localmente a un ROOT que sumará esos valores para encontrar el total de pares de primos gemelos en el rango [1,n]
 
2. Tomar el tiempo que le lleva a p procesos determinar cuántos primos hay en el rango dado. Hacer que p cambie de valor en el rango [1,15]. Con esto se genera una serie de datos que te servirán para dibujar una curva del  tiempo vs el numero de procesos

VERSION 2

1. Utilizar la función MPI_Reduce(), en lugar del envío/recepción de mensajes, para hacer el conteo total de pares de primos gemelos. 
Tomar tiempos y graficar haciendo que p tome valores en [1,15] (usar la misma gráfica que en la versión1)

    Hacer una versión secuencial que cuente el número de parejas de primos gemelos en el rango dado. Incluir el tiempo de la versión secuencial en la gráfica.


Evidencia:
    Programa secuencial
    Programa paralelo versión1
    Programa paralelo versión2
    BATCH
    Gráfica (imagen)

# Version Paralelo 1 

In [ ]:
%%writefile numPrimGemelos02.cpp
#include <iostream>
#include <cstdlib>
#include "mpi.h"

using std::cout;
using std::endl;

#define BLOCK_LOW(id,p,n) ((id)*(n)/(p))
#define BLOCK_HIGH(id,p,n) (BLOCK_LOW((id)+1,p,n)-1)

int main(int argc, char **argv ) {      
  MPI_Init( &argc , &argv); 
  int id, i, j, np, n, gap, inicio, fin, root = 0;

  MPI_Comm_rank(MPI_COMM_WORLD, &id); 
  MPI_Comm_size(MPI_COMM_WORLD, &np); 

  n      = atoi(argv[1]);
  inicio = BLOCK_LOW(id, np, n) + 1;
  fin    = BLOCK_HIGH(id, np, n) + 1;

  int numPrimGemelos = 0, primerPrimAbs = 0, primerPrimo = 0, ultimoPrimo = 0;
  
  // Comenzamos la medición del tiempo.
  MPI_Barrier(MPI_COMM_WORLD);
  double inicioTiempo = MPI_Wtime();

  for (int i = (inicio == 1 ? 2 : inicio); i <= fin; i++) {
      bool esPrimo = true;

      for (int j = 2; j < i; j++) {
          if (i % j == 0) {
              esPrimo = false;

              break;
          }
      }
      
      if (esPrimo) {
          if (primerPrimo == 0) {
              primerPrimAbs = primerPrimo = i;
          } else if (ultimoPrimo == 0) {
              ultimoPrimo = i;

              if (ultimoPrimo - 2 == primerPrimo) {
                  numPrimGemelos++;
              }
          } else {
              primerPrimo = ultimoPrimo;
              ultimoPrimo = i;

              if (ultimoPrimo - 2 == primerPrimo) {
                  numPrimGemelos++;
              }
          }
      }
  }

  if (ultimoPrimo == 0) {
      ultimoPrimo = primerPrimo;
  }

  if (id != np - 1 && ultimoPrimo != 0) {
      MPI_Send(&ultimoPrimo, 1, MPI_INT, id + 1, id * 10 + 1, MPI_COMM_WORLD);   
  }
  
  if (id != root) {
      int primoRecibido;
      MPI_Recv(&primoRecibido, 1, MPI_INT, id - 1, (id - 1) * 10 + 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

      if (primoRecibido > 0 && primoRecibido < primerPrimAbs) {
          if (primerPrimAbs - 2 == primoRecibido) {
              numPrimGemelos++;
          }
      }
      
  }
  
  int numPrimGemTotal;
  MPI_Send(&numPrimGemelos, 1, MPI_INT,root,9, MPI_COMM_WORLD);
  if(id == root) {
    for(i = 0; i<np;i++){
        MPI_Recv(&numPrimGemelos, 1, MPI_INT,i,9, MPI_COMM_WORLD,MPI_STATUS_IGNORE);
        numPrimGemTotal += numPrimGemelos;
    }
    // Finalizamos la medición del tiempo.
    double finTiempo = MPI_Wtime();
    double tiempoTotal = finTiempo - inicioTiempo;  
    
    cout << "\nRango (1, " << n << ")" << endl;
    cout << "Cantidad de Procesadores: " << np << endl;
    cout << "Cantidad de Primos Gemelos: " << numPrimGemTotal << endl;
    cout << "Tiempo Total: " << tiempoTotal << endl;
  }
  
  MPI_Finalize(); 
}



Overwriting numPrimGemelos02.cpp


In [ ]:
%%script bash

# !/bin/bash
# SBATCH --nodes=1                          #numero de nodos (maximo 13)
# SBATCH --job-name=MPI                     #nombre del job
# SBATCH --ntasks-per-node=15
# SBATCH --time=1:00:00                     #tiempo de ejecucion (maximo 168 hrs)
# SBATCH --partition=general                #cola de slurm
# SBATCH --constraint=broadwell

module load intel/oneAPI-2021

mpicxx numPrimGemelos02.cpp -o version1
max_procesadores=2
 
for((i=1;i<=$max_procesadores;++i)); do
  mpirun -np $i ./version1 datos.dat >>tiempos1.dat
done

%%script bash

# !/bin/bash
# SBATCH --nodes=1                          #numero de nodos (maximo 13)
# SBATCH --job-name=MPI                     #nombre del job
# SBATCH --ntasks-per-node=15
# SBATCH --time=1:00:00                     #tiempo de ejecucion (maximo 168hrs)
# SBATCH --partition=general                #cola de slurm
# SBATCH --constraint=broadwell

module load intel/oneAPI-2021
mpicxx encontrarGap.cpp -o max_gap

mpirun --allow-run-as-root -np 10 ./max_gap 2000000 

bash: line 10: module: command not found
--------------------------------------------------------------------------
mpirun has detected an attempt to run as root.
Running at root is *strongly* discouraged as any mistake (e.g., in
defining TMPDIR) or bug can result in catastrophic damage to the OS
file system, leaving your system in an unusable state.

You can override this protection by adding the --allow-run-as-root
option to your cmd line. However, we reiterate our strong advice
against doing so - please do so at your own risk.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
mpirun has detected an attempt to run as root.
Running at root is *strongly* discouraged as any mistake (e.g., in
defining TMPDIR) or bug can result in catastrophic damage to the OS
file system, leaving your system in an unusable state.

You can override this protection by adding the --allow-run-as-root
option to

CalledProcessError: ignored

In [ ]:
%%writefile numPrimGemelos01.cpp

#include <iostream>
#include <cstdlib>
#include "mpi.h"

using std::cout;
using std::endl;

#define BLOCK_LOW(id,p,n) ((id)*(n)/(p))
#define BLOCK_HIGH(id,p,n) (BLOCK_LOW((id)+1,p,n)-1)

int main(int argc, char **argv ) {      
  MPI_Init( &argc , &argv); 
  int id, i, j, np, n, gap, inicio, fin, root = 0;

  MPI_Comm_rank(MPI_COMM_WORLD, &id); 
  MPI_Comm_size(MPI_COMM_WORLD, &np); 

  n      = atoi(argv[1]);
  inicio = BLOCK_LOW(id, np, n) + 1; // para que empiece en 1
  fin    = BLOCK_HIGH(id, np, n) + 1;

  int numPrimGemelos = 0, primerPrimAbs = 0, primerPrimo = 0, ultimoPrimo = 0;

  // Comenzamos la medición del tiempo.
  MPI_Barrier(MPI_COMM_WORLD);
  double inicioTiempo = MPI_Wtime();

  for (int i = (inicio == 1 ? 2 : inicio); i <= fin; i++) {
      bool esPrimo = true;

      for (int j = 2; j < i; j++) {
          if (i % j == 0) {
              esPrimo = false;

              break;
          }
      }
      
      if (esPrimo) {
          if (primerPrimo == 0) {
              primerPrimAbs = primerPrimo = i;
          } else if (ultimoPrimo == 0) {
              ultimoPrimo = i;

              if (ultimoPrimo - 2 == primerPrimo) {
                  numPrimGemelos++;
              }
          } else {
              primerPrimo = ultimoPrimo;
              ultimoPrimo = i;

              if (ultimoPrimo - 2 == primerPrimo) {
                  numPrimGemelos++;
              }
          }
      }
  }

  if (ultimoPrimo == 0) {
      ultimoPrimo = primerPrimo;
  }

  if (id != np - 1 && ultimoPrimo != 0) {
      // Enviar ultimoPrimo con SEND().
      MPI_Send(&ultimoPrimo, 1, MPI_INT, id + 1, id * 10 + 1, MPI_COMM_WORLD);   
  }
  
  if (id != root) {
      int primoRecibido;
      MPI_Recv(&primoRecibido, 1, MPI_INT, id - 1, (id - 1) * 10 + 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

      if (primoRecibido > 0 && primoRecibido < primerPrimAbs) {
          if (primerPrimAbs - 2 == primoRecibido) {
              numPrimGemelos++;
          }
      }
      
  }
  
  int numPrimGemTotal;
  MPI_Reduce(&numPrimGemelos, &numPrimGemTotal, 1, MPI_INT, MPI_SUM, root, MPI_COMM_WORLD);

  if(id == root) {
    // Finalizamos la medición del tiempo.
    double finTiempo = MPI_Wtime();
    double tiempoTotal = finTiempo - inicioTiempo;  
    
  //cout << "\nRango (1, " << n << ")" << endl;
    cout << np <<',' << tiempoTotal <<  endl;
  //cout << "Cantidad de Primos Gemelos: " << numPrimGemTotal << endl;
  //  cout << "Tiempo Total: " << tiempoTotal << endl;
  }
 
  MPI_Finalize(); 
}

Writing numPrimGemelos01.cpp


#Version Secuencial

In [ ]:
%%writefile numPrimGemelos02.cpp

#include <iostream>
#include <cstdlib>
#include <sys/time.h>

using std::cout;
using std::endl;

int main(int argc, char **argv ) {      
  int numPrimGemelos = 0, primerPrimo = 0, ultimoPrimo = 0, finRango = atoi(argv[1]);

  // Comenzamos la medición del tiempo.
  struct timeval inicioTiempo, finTiempo;
  gettimeofday(&inicioTiempo, 0);

  for (int i = 1; i <= finRango; i++) {
      bool esPrimo = true;

      for (int j = 2; j < i; j++) {
          if (i % j == 0) {
              esPrimo = false;

              break;
          }
      }
      
      if (esPrimo) {
          if (primerPrimo == 0) {
              primerPrimo = i;
          } else if (ultimoPrimo == 0) {
              ultimoPrimo = i;

              if (ultimoPrimo - 2 == primerPrimo) {
                  numPrimGemelos++;
              }
          } else {
              primerPrimo = ultimoPrimo;
              ultimoPrimo = i;

              if (ultimoPrimo - 2 == primerPrimo) {
                  numPrimGemelos++;
              }
          }
      }
  }

  // Finalizamos la medición del tiempo.
  gettimeofday(&finTiempo, 0);

  long segundos = finTiempo.tv_sec - inicioTiempo.tv_sec;
  long microsegundos = finTiempo.tv_usec - inicioTiempo.tv_usec;
  double tiempoTotal = segundos + (microsegundos * 1e-6);

  cout << "\nRango (1, " << finRango << ")" << endl;
  cout << "Cantidad de Primos Gemelos: " << numPrimGemelos << endl;
  cout << "Tiempo Total: " << tiempoTotal << endl;
}

Writing numPrimGemelos02.cpp


In [ ]:
%%script bash

g++ numPrimGemelos02.cpp -o cantPrimGemelos02

./cantPrimGemelos02 100


Rango (1, 100)
Cantidad de Primos Gemelos: 8
Tiempo Total: 6e-06


In [ ]:
%%writefile generador_archivo_datos.c
#include<stdio.h>
#include<stdlib.h>
#include<time.h>
// ---> argv[1]: Nombre del archivo a generar <---
// ---> argv[2]: numero de datos a generar    <---
int main(int argc, char*argv[]){
  FILE *fp;
  int n,*datos, i;
  
  n = atoi(argv[2]);
  fp = fopen(argv[1], "w");
  if(fp){
    srand(time(NULL));
    fprintf(fp,"%d\n",n);
    for(i=0;i<n;i++)
      fprintf(fp,"%d\n",rand()%1000);
    
    fclose(fp);
  }
  
	return 0;
}

In [ ]:
%%script bash
gcc generador_archivo_datos.c -o generar
./generar datos.dat 200000

#Tarea 4 :Máximo intervalo entre números primos

La distancia (gap) entre los números primos consecutivos 2 y 3 es 1, mientras que el gap entre el 7 y el 11 es 4.
Escribir un programa paralelo para determinar para todos los enteros menores que 2,000,000, el gap mas grande entre un par de números primos consecutivos.

Comparar contra una versión secuencial, usando p=1..15, y obtener las gráficas de tiempo, speedup y eficiencia.


Evidencia:
    Programa secuencial
    Programa paralelo versión1
    Programa paralelo versión2
    BATCH
    Gráfica (imagen)

In [ ]:
%%writefile encontrarGap.cpp

#include <iostream>
#include <cstdlib>
#include "mpi.h"

using std::cout;
using std::endl;

#define BLOCK_LOW(id,p,n) ((id)*(n)/(p))
#define BLOCK_HIGH(id,p,n) (BLOCK_LOW((id)+1,p,n)-1)
#define BLOCK_SIZE(id,p,n)(BLOCK_HIGH(id,p,n)-BLOCK_LOW(id,p,n)+1)

int main(int argc, char **argv ) {      
  MPI_Init( &argc , &argv); 
  int id, i, j, np, n, gap, inicio, fin, num_elems_rango, root = 0;

  MPI_Comm_size(MPI_COMM_WORLD, &np);
  MPI_Comm_rank(MPI_COMM_WORLD, &id);

  n      = atoi(argv[1]);
  inicio = BLOCK_LOW(id, np, n) + 1; // para que empiece en 1
  fin    = BLOCK_HIGH(id, np, n) + 1;
  num_elems_rango = BLOCK_SIZE(id, np, n);

  int maxGap = 0, primerPrimAbs = 0, primerPrimo = 0, ultimoPrimo = 0;

  cout << "numProc_" << id << ", rango_" << inicio <<','<< fin << endl;

  // Comenzamos la medición del tiempo.
  MPI_Barrier(MPI_COMM_WORLD);
  double inicioTiempo = MPI_Wtime();

  for (int i = (inicio == 1 ? 2 : inicio); i <= fin; i++) {
      bool esPrimo = true;

      for (int j = 2; j < i; j++) {
          if (i % j == 0) {
              esPrimo = false;

              break;
          }
      }
      
      if (esPrimo) {
          if (primerPrimo == 0) {
              primerPrimAbs = primerPrimo = i;
          } else if (ultimoPrimo == 0) {
              ultimoPrimo = i;

              maxGap = ultimoPrimo - primerPrimo;
          } else {
              primerPrimo = ultimoPrimo;
              ultimoPrimo = i;

              if (maxGap < ultimoPrimo - primerPrimo) {
                  maxGap = ultimoPrimo - primerPrimo;
              }
          }
      }
  }

  if (ultimoPrimo == 0) {
      ultimoPrimo = primerPrimo;
  }

  if (id != np - 1 && ultimoPrimo != 0) {
      // Enviar ultimoPrimo con SEND().
      MPI_Send(&ultimoPrimo, 1, MPI_INT, id + 1, id * 10 + 1, MPI_COMM_WORLD);   
  }
  
  if (id != root) {
      int primoRecibido;
      MPI_Recv(&primoRecibido, 1, MPI_INT, id - 1, (id - 1) * 10 + 1, MPI_COMM_WORLD, MPI_STATUS_IGNORE);

      if (primoRecibido > 1 && primoRecibido < primerPrimAbs) {
          if (maxGap < primerPrimAbs - primoRecibido) {
              maxGap = primerPrimAbs - primoRecibido;
          }
      }
      
  }
  
  int maxGapFinal;
  MPI_Reduce(&maxGap, &maxGapFinal, 1, MPI_INT, MPI_MAX, root, MPI_COMM_WORLD);

  if(id == root) {
    // Finalizamos la medición del tiempo.
    double finTiempo = MPI_Wtime();
    double tiempoTotal = finTiempo - inicioTiempo;  
    
    cout << "\nRango (1, " << n << ")" << endl;
    cout << "Cantidad de Procesadores: " << np << endl;
    cout << "Max Gap: " << maxGapFinal << endl;
    cout << "Tiempo Total: " << tiempoTotal << endl;
  }
 
  MPI_Finalize(); 
} 

Writing encontrarGap.cpp


In [ ]:
%%script bash

# !/bin/bash
# SBATCH --nodes=1                          #numero de nodos (maximo 13)
# SBATCH --job-name=MPI                     #nombre del job
# SBATCH --ntasks-per-node=15
# SBATCH --time=1:00:00                     #tiempo de ejecucion (maximo 168hrs)
# SBATCH --partition=general                #cola de slurm
# SBATCH --constraint=broadwell

module load intel/oneAPI-2021
mpicxx encontrarGap.cpp -o max_gap

mpirun --allow-run-as-root -np 10 ./max_gap 2000000 

#!/bin/bash
#SBATCH --nodes=1                          #numero de nodos (maximo 13)
#SBATCH --job-name=MPI                     #nombre del job
#SBATCH --ntasks-per-node=15
#SBATCH --time=1:00:00                     #tiempo de ejecucion (maximo 168hrs)
#SBATCH --partition=general                #cola de slurm
#SBATCH --constraint=broadwell
module load intel/oneAPI-2021
mpicxx gapsec.cpp -o gapsec -lm
 mpirun  -np 1 ./gapsec 200000 



#!/bin/bash
#SBATCH --nodes=1                          #numero de nodos (maximo 13)
#SBATCH --job-name=MPI                     #nombre del job
#SBATCH --ntasks-per-node=15
#SBATCH --time=1:00:00                     #tiempo de ejecucion (maximo 168 hrs)
#SBATCH --partition=general                #cola de slurm
#SBATCH --constraint=broadwell
module load intel/oneAPI-2021
mpicxx primosgem2.cpp -o primosgem2
max_procesadores=15
for((i=1;i<=$max_procesadores;++i)); do
  mpirun -np $i ./primosgem2 2000000 >>tiempos1.dat
done





Maxgap.cpp




numProc_5, rango_1000001,1200000
numProc_6, rango_1200001,1400000
numProc_0, rango_1,200000
numProc_7, rango_1400001,1600000
numProc_8, rango_1600001,1800000
numProc_1, rango_200001,400000
numProc_9, rango_1800001,2000000
numProc_3, rango_600001,800000
numProc_2, rango_400001,600000
numProc_4, rango_800001,1000000

Rango (1, 2000000)
Cantidad de Procesadores: 10
Max Gap: 132
Tiempo Total: 323.101


In [ ]:
%%writefile numPrimGemelos02.cpp

#include <iostream>
#include <cstdlib>
#include <sys/time.h>

using std::cout;
using std::endl;

int main(int argc, char **argv ) {      
  int numPrimGemelos = 0, primerPrimo = 0, ultimoPrimo = 0, finRango = atoi(argv[1]);

  // Comenzamos la medición del tiempo.
  struct timeval inicioTiempo, finTiempo;
  gettimeofday(&inicioTiempo, 0);

  for (int i = 1; i <= finRango; i++) {
      bool esPrimo = true;

      for (int j = 2; j < i; j++) {
          if (i % j == 0) {
              esPrimo = false;

              break;
          }
      }
      
      if (esPrimo) {
          if (primerPrimo == 0) {
              primerPrimo = i;
          } else if (ultimoPrimo == 0) {
              ultimoPrimo = i;

              if (ultimoPrimo - 2 == primerPrimo) {
                  numPrimGemelos++;
              }
          } else {
              primerPrimo = ultimoPrimo;
              ultimoPrimo = i;

              if (ultimoPrimo - 2 == primerPrimo) {
                  numPrimGemelos++;
              }
          }
      }
  }

  // Finalizamos la medición del tiempo.
  gettimeofday(&finTiempo, 0);

  long segundos = finTiempo.tv_sec - inicioTiempo.tv_sec;
  long microsegundos = finTiempo.tv_usec - inicioTiempo.tv_usec;
  double tiempoTotal = segundos + (microsegundos * 1e-6);

  cout << "\nRango (1, " << finRango << ")" << endl;
  cout << "Cantidad de Primos Gemelos: " << numPrimGemelos << endl;
  cout << "Tiempo Total: " << tiempoTotal << endl;
}


In [ ]:
#!/bin/bash

# PBS Job Script para uso de MPI

#Modifique segun el numero de procesadores necesitados

#PBS -l nodes=1:ppn=4
#PBS -l walltime=00:10:00
#PBS -N MPI
#PBS -V
#PBS -q batch
#PBS -k eo

## source /opt/intel/oneapi/setvars.sh

mpicxx gap02.cpp -o max_gap_02

mpirun  -np 1 ./max_gap_02 2000000 >> sec.data


#Generador de datos 

In [ ]:
%%writefile generador_archivo_datos.c
#include<stdio.h>
#include<stdlib.h>
#include<time.h>
// ---> argv[1]: Nombre del archivo a generar <---
// ---> argv[2]: numero de datos a generar    <---
int main(int argc, char*argv[]){
  FILE *fp;
  int n,*datos, i;
  
  n = atoi(argv[2]);
  fp = fopen(argv[1], "w");
  if(fp){
    srand(time(NULL));
    fprintf(fp,"%d\n",n);
    for(i=0;i<n;i++)
      fprintf(fp,"%d\n",rand()%1000);
    
    fclose(fp);
  }
  
	return 0;
}

Writing generador_archivo_datos.c


In [ ]:
%%script bash
gcc generador_archivo_datos.c -o generar
./generar datos.dat 5000000

#Tarea 5: Merge-split Sort
1. Generar un archivo de datos (datos.dat), con 10 millones de números a ordenar. Puedes hacer pruebas con más datos.
Hacer una versión secuencial de ordenamiento, con quicksort, para una comparación justa (es el más rápido). 

2. Implementar el algoritmo en paralelo en el lenguaje de tu preferencia+MPI . 
Tomar los tiempos considerando [1,15] procesadores.
Generar las gráficas de tiempos, speedup y eficiencia.

In [ ]:
%%writefile ordenamientoQsort_PAR.cpp
#include <stdlib.h>
#include "mpi.h"
#include <stdio.h>
#include <math.h> // ceil()
#include <limits.h> // INT_MAX 

// Función utilizada por qsort() para comparar los elementos.
int comparar(const void *_a, const void *_b) {
    int *a, *b;
    
    a = (int *) _a;
    b = (int *) _b;
    
    return (*a - *b);
}

// Ordena los subarreglos resultantes.
void mezcla(int *S_i, int chunksize) {
    int *aux;

    aux = (int *)malloc(chunksize*sizeof(int));

    int i = 0, j = 0, k =  chunksize/2;

    while (j < chunksize/2 && k < chunksize) {
        aux[i++] = (S_i[j] < S_i[k]) ? S_i[j++] : S_i[k++];
    }

    while (j < chunksize/2) {
        aux[i++] = S_i[j++];
    }

    while (k < chunksize) {
        aux[i++] = S_i[k++];
    }

    for (i = 0; i < chunksize; i++) {
        S_i[i] = aux[i];
    }

    free(aux);
}

// Comprueba si se le ha asignado memoria al puntero.
void asignarMemoria(int **arreglo, const unsigned numElem) {
  *arreglo = (int *) malloc(numElem * sizeof(int));

  if (*arreglo == NULL) {
    printf("Memoria insuficiente\n");

    MPI_Abort(MPI_COMM_WORLD, 99);
  }
}

// Imprime el arreglo completo.
void imprimirArreglo(const int *arreglo, const unsigned numElem) {
	printf("[");
	
	for (int i = 0; i < numElem; i++) {
		printf("%d, ", arreglo[i]);
	}
	
	printf("\b\b]\n");
}

//  Lectura de archivos
int read_array(char* fname, int **arr, int np) {
  FILE *myFile;
  unsigned int i, n, chunksize, faltantes;

  myFile = fopen(fname, "r");
  if(!myFile){
    printf("ERROR: No se pudo abrir el archivo para lectura %s",fname);
    MPI_Abort(MPI_COMM_WORLD, 99);
  }

  fscanf(myFile, "%i\n", &n); // numero de datoss a leer

  chunksize = ceil(n*1.0 / np);
  faltantes = np*chunksize - n;

  asignarMemoria(arr, chunksize*np);



  for ( i=0; i < n; i++)
    fscanf(myFile, "%i\n", (*arr)+i);

  for ( i=n; i < n + faltantes; i++) //<--- Dummys
    (*arr)[i] = INT_MAX;

  return n;
}

// Programa principal
int main(int argc, char**argv) {
  int rank, np, root, faltantes;
  unsigned int n, chunksize, i;
  double start, time;
  int *Arreglo, *S_i;

  MPI_Init(&argc, &argv);

  MPI_Comm_size(MPI_COMM_WORLD, &np); // Numero total de procesos
  MPI_Comm_rank(MPI_COMM_WORLD, &rank); // Valor de nuestro identificador

  root = np - 1;  // Root es el último

  MPI_Barrier(MPI_COMM_WORLD);  // sincronización
  double tInicial = MPI_Wtime();  // <--- toma de tiempo 1

  if (rank == root){ // Lee los datos del archivo
    n = read_array(argv[1], &Arreglo, np);
    chunksize = ceil(n*1.0 / np);
    faltantes = np*chunksize - n;

    imprimirArreglo(Arreglo, n);

  }
  
  // Comunica longitud de los chunks
  MPI_Bcast(&chunksize, 1, MPI_UNSIGNED, root, MPI_COMM_WORLD);
  
  // Pide memoria del doble de dicha longitud para poder ordenar.
  asignarMemoria(&S_i, 2*chunksize);


  MPI_Scatter(Arreglo, chunksize, MPI_INT, S_i, chunksize, MPI_INT, root, MPI_COMM_WORLD);

  qsort(  S_i, chunksize, sizeof(int), &comparar );

  for(i=1;i<=ceil(np/2.0);i++) {
    if(rank%2==1) {
         MPI_Send( S_i, chunksize, MPI_INT, rank-1, 10, MPI_COMM_WORLD );
         MPI_Recv( S_i, chunksize, MPI_INT, rank-1, 10, MPI_COMM_WORLD, MPI_STATUS_IGNORE );
    }

    if(rank%2==0 && rank!=np-1) {
      MPI_Recv( S_i+chunksize, chunksize, MPI_INT, rank+1, 10, MPI_COMM_WORLD, MPI_STATUS_IGNORE );   
      mezcla(S_i, 2*chunksize); // <---- 
      MPI_Send(S_i+chunksize, chunksize, MPI_INT, rank+1, 10, MPI_COMM_WORLD);  
    }

    if(rank%2==0 && rank!=0) {
         MPI_Send( S_i, chunksize, MPI_INT, rank-1, 10, MPI_COMM_WORLD );
         MPI_Recv( S_i, chunksize, MPI_INT, rank-1, 10, MPI_COMM_WORLD, MPI_STATUS_IGNORE );
    }

    if(rank%2==1 && rank!=np-1) {
      MPI_Recv( S_i+chunksize, chunksize, MPI_INT, rank+1, 10, MPI_COMM_WORLD, MPI_STATUS_IGNORE );   
      mezcla(S_i, 2*chunksize); // <---- 
      MPI_Send(S_i+chunksize, chunksize, MPI_INT, rank+1, 10, MPI_COMM_WORLD);  
    }
  }
  
  // Recolectar de nuevo los datos en el root e imprimir
  MPI_Gather(S_i, chunksize, MPI_INT, Arreglo, chunksize, MPI_INT, root, MPI_COMM_WORLD );
  
  double tFinal = MPI_Wtime(); // <--- toma de tiempo 2

  if(rank==root) {
      imprimirArreglo(Arreglo, n);



      double tTotal = tFinal - tInicial; 

      printf("\nN\243mero de procesos = %d", np);
      printf("\nCantidad de elementos = %d", n);
      printf("\nTiempo total = %f segundos", tTotal);
      printf(" \n");
  }
  
  free(S_i);
  if (rank == root) free(Arreglo);

  MPI_Finalize();
  return 0;
}

In [ ]:
#!/bin/bash
#SBATCH --nodes=1                          #numero de nodos (maximo 13)
#SBATCH --job-name=MPI                     #nombre del job
#SBATCH --ntasks-per-node=15
#SBATCH --time=1:00:00                     #tiempo de ejecucion (maximo 168 hrs)
#SBATCH --partition=general                #cola de slurm
#SBATCH --constraint=broadwell
module load intel/oneAPI-2021

mpicxx mergesrt.cpp -o mergesrt.cpp -lm
max_procesadores=16

for((i=1;i<=$max_procesadores;++i)); do
  mpirun -np $i ./mergesrt datos.dat >> tiempos.dat
done

In [ ]:
#Ordenamiento secuencial
%%writefile ordenamientoQsort_SEC.c
#include <stdlib.h>
#include <stdio.h>
#include <math.h> // ceil()
#include <limits.h> // INT_MAX 
#include <sys/time.h>

// Función utilizada por qsort() para comparar los elementos.
int comparar(const void *_a, const void *_b) {
    int *a, *b;
    
    a = (int *) _a;
    b = (int *) _b;
    
    return (*a - *b);
}

void imprimirArreglo(const int *arreglo, const unsigned numElem) {
	printf("[");
	
	for (int i = 0; i < numElem; i++) {
		printf("%d, ", arreglo[i]);
	}
	
	printf("\b\b]\n");
}

// Lectura de los datos.
int read_array(char* fname, int **arr) {
  FILE *myFile;
  myFile = fopen(fname, "r");

  if (!myFile) {
    printf("ERROR: No se pudo abrir el archivo para lectura %s",fname);
    return -1;
  }

  unsigned int n;
  fscanf(myFile, "%i\n", &n); // Número de datos a leer.

  *arr = (int *) malloc(n*1.0*sizeof(int));

  if (*arr == NULL) {
    printf("Memoria insuficiente\n");
    return -1;
  }

  for (int i = 0; i < n; i++)
    fscanf(myFile, "%i\n", (*arr)+i);

  return n;
}

// Programa principal
int main(int argc, char**argv) {
  int *Arreglo;
  unsigned int n;

  struct timeval tInicial;  // <--- toma de tiempo 1
  gettimeofday(&tInicial, 0);
  
  n = read_array(argv[1], &Arreglo);

  imprimirArreglo(Arreglo, n);

  qsort(Arreglo, n, sizeof(int), &comparar);
 
  struct timeval tFinal;  // <--- toma de tiempo 2
  gettimeofday(&tFinal, 0);

  long segundos = tFinal.tv_sec - tInicial.tv_sec;
  long microsegundos = tFinal.tv_usec - tInicial.tv_usec;
  double tTotal = segundos + (microsegundos * 1e-6);

  printf("\nCantidad de elementos = %d", n);
  printf("\nTiempo total = %f segundos", tTotal);

  free(Arreglo);

  return 0;
}

In [ ]:
%%script bash
g++ ordenamientoQsort_SEC.cpp -o Ordenamiento_SEC
./Ordenamiento_SEC datos.dat

#Tarea 6:  Ordenamiento Regular Sampling- Paralelo

In [ ]:
%%writefile funciones.h
#include <stdlib.h>
#include <stdio.h>
#include <math.h> // ceil()
#include <limits.h> // INT_MAX
#include "mpi.h"

#define BLOCK_LOW(id,p,n) ((id)*(n)/(p))
#define BLOCK_HIGH(id,p,n) (BLOCK_LOW((id)+1,p,n)-1)
#define BLOCK_SIZE(id,p,n)(BLOCK_HIGH(id,p,n)-BLOCK_LOW(id,p,n)+1)

// Compara dos valores para la función qSort.
int comparar(const void *_a, const void *_b) {
	int *a, *b;
	
	a = (int *) _a;
	b = (int *) _b;
	
	return (*a - *b);
}

// Comprueba si se le ha asignado memoria al puntero.
void asignarMemoria(int **arreglo, const unsigned numElem, const char nombreVar[30]) {
	*arreglo = (int *) malloc(numElem * sizeof(int));
	
	if (*arreglo == NULL) {
		printf("Memoria insuficiente %s\n", nombreVar);
		
		MPI_Abort(MPI_COMM_WORLD, 99);
	}
}

//  Lectura de archivos
int read_array(char* fname, int **arr, int np) {
	FILE *myFile;
	myFile = fopen(fname, "r");
	
	if (!myFile) {
		printf("ERROR: No se pudo abrir el archivo para lectura %s", fname);

		MPI_Abort(MPI_COMM_WORLD, 99);
	}
	
	unsigned int n;
	fscanf(myFile, "%i\n", &n); // numero de datos a leer
	
	asignarMemoria(arr, n, "(arr)");
	
  for (int i = 0; i < n; i++) {
		fscanf(myFile, "%i\n", (*arr) + i);
	}
	
	for (int i = n; i < n; i++) {
		fscanf(myFile, "%i\n", (*arr)+i);
	}
	
	return n;
}

// Imprime el arreglo completo.
void imprimirArreglo(const int *arreglo, const unsigned numElem) {
	printf("[");
	
	for (int i = 0; i < numElem; i++) {
		printf("%d, ", arreglo[i]);
	}
	
	printf("\b\b]\n");
}

// Ordena los elementos del arreglo dado.
void mezclar(int *destino, int *numDestino, int *agregado, int numAgregado) {
    int numAux = (*numDestino) + numAgregado;
		int *aux;

		asignarMemoria(&aux, numAux, "(aux)");

    int i = 0, j = 0, k = 0;
    while (j < (*numDestino) && k < numAgregado) {
        aux[i++] = (destino[j] < agregado[k]) ? destino[j++] : agregado[k++];
    }

    while (j < (*numDestino)) {
        aux[i++] = destino[j++];
    }

    while (k < numAgregado) {
        aux[i++] = agregado[k++];
    }

    for (i = 0; i < numAux; i++) {
        destino[i] = aux[i];
    }

		(*numDestino) = numAux;

    free(aux);
}

In [ ]:
%%writefile regularSampling.c
#include "funciones.h"

int main(int argc, char**argv) {
	int id, np, n, i, j, n_local, ndatos_recv;
	int *pivotes, *sendcounts, *sendbuf, *recvcounts, *recvbuf, *recvbuf_mezclas;
	int *muestras , *todas_las_muestras, *displs, *sdispls, *rdispls;
	double start, time;
	
	MPI_Init(&argc, &argv);
	MPI_Comm_size(MPI_COMM_WORLD, &np); // Numero total de procesos
	MPI_Comm_rank(MPI_COMM_WORLD, &id); // Valor de nuestro identificador
	
	MPI_Barrier(MPI_COMM_WORLD);  // sincronización
	start = MPI_Wtime();
	
	// El root lee un dataset del archivo con n datos e inicia una dispersión de los datos.  <-----  (1)
	int root = np - 1;
	if (id == root) { // Lee los datos del archivo
		n = read_array(argv[1], &sendbuf, np);
  }
	
	asignarMemoria(&sendcounts, np, "(sendcounts)");
	
	if (id == root) {
		for (i = 0; i < np; i++) {
			sendcounts[i] = BLOCK_SIZE(i, np, n);
		}
		
		asignarMemoria(&displs, np, "(displs)");
		
		displs[0] = 0;
		for (i = 1; i < np; i++) {
			displs[i] = displs[i - 1] + sendcounts[i - 1];
		}
	}
	
	// Comunica longitud de los chunks
	MPI_Bcast(&n, 1, MPI_INT, root, MPI_COMM_WORLD);
	n_local = BLOCK_SIZE(id, np, n);
	
	asignarMemoria(&recvbuf, n_local, "(recvbuf)");
	
	MPI_Scatterv(sendbuf, sendcounts, displs, MPI_INT, recvbuf, n_local, MPI_INT, root, MPI_COMM_WORLD);
	
	// Cada proceso ordena localmente sus datos con QuickSort  <-----  (2)
	qsort(recvbuf, n_local, sizeof(recvbuf[0]), &comparar);
	
	// Cada proceso selecciona los n índices 0, n/p^2, 2n/p^2, 3n/p^2,..., (p-1)n/p^2 como muestras regulares <-----  (3)
	asignarMemoria(&muestras, np, "(muestras)");
	
	for (i = 0; i < np; i++) {
		muestras[i] = recvbuf[ i*n/(np*np) ];
	}

  // Un proceso recolecta y ordena las muestras regulares y selecciona p-1 valores pivote de la lista ordenada. <-----  (4)
  // Los valores pivote están en los índices p+p/2-1, 2p+p/2-1, 3p+p/2-1, ... , (p-1)p+p/2-1.
  // Los valores pivote son dados a conocer a todos los procesos.
	if (id == root) {
		asignarMemoria(&todas_las_muestras, np*np, "(todas_las_muestras)");
	}

	MPI_Gather( muestras, np, MPI_INT, todas_las_muestras, np, MPI_INT, root, MPI_COMM_WORLD );
	
	if (id == root) {
		qsort( todas_las_muestras, np*np, sizeof(int), &comparar );
	}

	asignarMemoria(&pivotes, np-1, "(pivotes)");

	if (id == root) {
		for (i = 0; i < np - 1; i++) {
			pivotes[i] = todas_las_muestras[ (i+1)*np + np/2 -1 ];
		}
	}

	MPI_Bcast(pivotes, np-1, MPI_INT, root, MPI_COMM_WORLD);

	// Cada proceso particiona su sublista ordenada en p piezas disjuntas, usando los valores pivote como separadores. <----- (5)
	for (i = 0, j = 0; i < (np-1); i++) {
		sendcounts[i] = 0;
		
		while (recvbuf[j] <= pivotes[i] && j < n_local) {
			sendcounts[i]++;
			
			j++;
		}
	}
	
	sendcounts[np-1] = 0;
	while (j < n_local) {
		sendcounts[np-1]++;
		
		j++;
	}

	asignarMemoria(&sdispls, np, "(sdispls)");

	sdispls[0] = 0;
	for (i = 1; i < np; i++) {
		sdispls[i] = sdispls[i-1] + sendcounts[i-1];
	}

	asignarMemoria(&recvcounts, np, "(recvcounts)");

	MPI_Alltoall( sendcounts, 1, MPI_INT, recvcounts, 1, MPI_INT, MPI_COMM_WORLD );
	
	asignarMemoria(&rdispls, np, "(rdispls)");

	rdispls[0] = 0;
	for (i = 1; i < np; i++) {
		rdispls[i] = rdispls[i-1] + recvcounts[i-1];
	}

	ndatos_recv = 0;
	for (i = 0; i < np; i++) {
		ndatos_recv += recvcounts[i];
	}
	
	asignarMemoria(&recvbuf_mezclas, ndatos_recv, "(recvbuf_mezclas)");

	// Cada proceso P_i mantiene su i-ésima partición y envía la j-ésima partición al proceso P_j (alltoallv)  <----- (6)
	MPI_Alltoallv( recvbuf, sendcounts, sdispls, MPI_INT, recvbuf_mezclas, recvcounts, rdispls, MPI_INT, MPI_COMM_WORLD );

	// Cada proceso mezcla sus p particiones para obtener una sola lista.  <----- (7)
	int cantDatos = recvcounts[0];
	for (i = 1; i < np; i++) {
			mezclar(recvbuf_mezclas, &cantDatos, recvbuf_mezclas + rdispls[i], recvcounts[i]);
	}

	// Recolectar los elementos que tiene cada proceso en un solo proceso con fines de impresión <---- (8)
	MPI_Gather(&ndatos_recv, 1, MPI_INT, recvcounts, 1, MPI_INT, root, MPI_COMM_WORLD);
	
	if (id == root) {
		rdispls[0] = 0;
		for (i = 1; i < np; i++) {
			rdispls[i] = rdispls[i-1] + recvcounts[i-1];
		}
	}
	
	MPI_Gatherv( recvbuf_mezclas, ndatos_recv, MPI_INT, sendbuf, recvcounts, rdispls, MPI_INT, root, MPI_COMM_WORLD );
	
	time = MPI_Wtime() - start;

  if (id == root) {
			printf("\nNúmero de procesos = %d", np);
			printf("\nCantidad de elementos = %d", n);
			printf("\nTiempo necesario = %f\n", time);
	}

	// Liberar memoria
	if (id == root) {
		free(sendcounts);
		free(displs);
		free(todas_las_muestras);
	}

	free(recvbuf);
	free(muestras);
	free(pivotes);
 
	MPI_Finalize();
	return 0;
}

In [ ]:
%%script bash

mpicxx regularSampling.c -o ordenRegSam -lm
mpirun --allow-run-as-root -np 5 ordenRegSam datos.dat 

#!/bin/bash
#SBATCH --nodes=1                          #numero de nodos (maximo 13)
#SBATCH --job-name=MPI                     #nombre del job
#SBATCH --ntasks-per-node=15
#SBATCH --time=1:00:00                     #tiempo de ejecucion (maximo 168 hrs)
#SBATCH --partition=general                #cola de slurm
#SBATCH --constraint=broadwell
module load intel/oneAPI-2021
mpicxx regularSampling.c -o regularSampling
max_procesadores=16
for((i=1;i<=$max_procesadores;++i)); do
	mpirun -np $i ./regularSampling datos.dat >>tiempos1.dat
done
